In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

In [82]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 10
batch_size = 128
display_step = 10

In [3]:
# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

In [85]:
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [86]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [4]:
#pred = RNN(x, weights, biases)

In [ ]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [5]:
# Initializing the variables
init = tf.global_variables_initializer()

In [6]:
training_iters = 500000

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

In [4]:
import numpy as np
from tensorflow.contrib.rnn import BasicLSTMCell
from tensorflow.python.ops.rnn import dynamic_rnn

In [5]:
# set up fake training data for a single sequence - n_examples X n_features
n_examples = 100
n_features = 373
n_categories = 1

In [6]:
data = np.random.normal(0,1,(n_examples,n_features))
labels = np.zeros(100)
for i in range(100):
    if i % 4 == 0 and i > 0:
        labels[i] = 1
        data[i] = (data[i-2]+data[i-1]/2)
labels

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.])

In [7]:
data.shape

(100, 373)

In [8]:
labels.shape

(100,)

In [9]:
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_categories])

In [10]:
seq_length = tf.placeholder(tf.int32)

In [12]:
n_hidden=500

In [13]:
learning_rate = 0.1
dropout = 0.5 

In [14]:
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_categories])),
    'out_bw': tf.Variable(tf.random_normal([n_hidden, n_categories]))
}

In [15]:
biases = {
    'out': tf.Variable(tf.random_normal([n_categories]))
}

In [16]:
number_of_layers=1

In [17]:
x.shape

TensorShape([Dimension(None), Dimension(373)])

In [18]:
"""
def network(x, my_weights, my_biases, dropout, seq_length):
    fc1 = tf.reshape(x, [1, -1, n_features])
    lstm_fw_cell = BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    (output_fw, state_fw) = tf.nn.dynamic_rnn(lstm_fw_cell,fc1,dtype='float32', sequence_length=tf.reshape(seq_length, [1]))
    outputs_fw = tf.reshape(output_fw, [seq_length, n_hidden])
    out = tf.matmul(outputs_fw, weights['out']) + biases['out']
    return out
"""

"\ndef network(x, my_weights, my_biases, dropout, seq_length):\n    fc1 = tf.reshape(x, [1, -1, n_features])\n    lstm_fw_cell = BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)\n    (output_fw, state_fw) = tf.nn.dynamic_rnn(lstm_fw_cell,fc1,dtype='float32', sequence_length=tf.reshape(seq_length, [1]))\n    outputs_fw = tf.reshape(output_fw, [seq_length, n_hidden])\n    out = tf.matmul(outputs_fw, weights['out']) + biases['out']\n    return out\n"

In [19]:
fc10 = tf.reshape(x, [1, -1, n_features])
fc10.shape

TensorShape([Dimension(1), Dimension(None), Dimension(373)])

In [20]:
#def network(x, weights, biases, dropout, seq_length):
fc1 = tf.reshape(x, [1, -1, n_features])

lstm_fw_cell = BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
lstm_fw_multicell = tf.contrib.rnn.MultiRNNCell([lstm_fw_cell] * number_of_layers, state_is_tuple=True)

lstm_bw_cell = BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
lstm_bw_multicell = tf.contrib.rnn.MultiRNNCell([lstm_bw_cell] * number_of_layers, state_is_tuple=True)

(output_fw, output_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(lstm_fw_multicell, lstm_bw_multicell, fc1, dtype='float32', sequence_length=tf.reshape(seq_length, [1]))

outputs_fw = tf.reshape(output_fw, [seq_length, n_hidden])
outputs_bw = tf.reshape(output_bw, [seq_length, n_hidden])

out = tf.matmul(outputs_fw, weights['out']) + tf.matmul(outputs_bw, weights['out_bw']) + biases['out']
#return out

In [21]:
#pred = network(x, weights, biases, dropout, seq_length)
pred = out

In [22]:
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y))

In [40]:
optimizer = tf.train.AdamOptimizer().minimize(cost)#(learning_rate=0.1, l2_regularization_strength=0.1).minimize(cost)

In [41]:
prediction = tf.sigmoid(pred)

In [42]:
correct_pred = tf.equal(tf.greater_equal(prediction,0.5), tf.equal(y,1))

In [43]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [44]:
#prediction=tf.argmax(pred,1)
#correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [45]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [46]:
sess = tf.Session()
sess.run(init)

In [47]:
#with tf.Session() as sess:
#    sess.run(init)
for i in range(100):
    acc, _ = sess.run([accuracy, optimizer], feed_dict={x:data,y:labels.reshape(100,1),seq_length:100})
    print(acc)
        #o = sess.run(out, feed_dict={x:data,y:labels.reshape(100,1),seq_length:100})
    #print(o)
    #sess.run(optimizer, feed_dict={x:data,y:labels.reshape(100,1),seq_length:100})
    #sess.run(sequence_length)
    #acc = sess.run(accuracy)
    #print(acc)

0.43
0.72
0.74
0.72
0.77
0.84
0.87
0.85
0.81
0.85
0.97
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [35]:
data.shape

(100, 373)

In [44]:
sess.run(state_fw, feed_dict={x:data[8:16],y:labels[8:16].reshape(8,1),seq_length:8})[0][1]

array([[-0.05282238,  0.17251636,  0.43274066, -0.11777841, -0.31262901,
         0.16178201, -0.30558801,  0.14845403,  0.34887356, -0.17319676,
         0.19785005, -0.04521424, -0.08704349,  0.16558027,  0.24718714,
        -0.24618267, -0.12641269, -0.12969455,  0.192828  , -0.10095158,
         0.15629388,  0.13893135,  0.06724413, -0.28869233,  0.34881324]], dtype=float32)

In [22]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    sess.run(optimizer, feed_dict={x:data,y:labels.reshape(100,1)})
    
    
"""
    b
    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
"""

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype int32
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_INT32, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: _recv_Placeholder_0 = _Recv[client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=3363666453783569408, tensor_name="Placeholder:0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder_2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch